In [1]:
import pandas as pd
import os
import fnmatch
import timeit

In [9]:
# df_left = pd.DataFrame.from_dict({'ARTIST':['Cardi B, Bad Bunny & J Balvin','Ed Sheeran'], 'TITLE': ['I Like It','Shape of You']})
 
df_left = pd.DataFrame.from_dict({'ARTIST':['adele'], 'TITLE': ['hello']})

In [10]:
df_left

,ARTIST,TITLE
0,adele,hello


In [7]:
df_left=pd.read_csv('../verified_lyrics_2.csv')
df_left.head()


,song_id,ISRC,SPOTIFY_ID,TITLE,ARTIST,verified_title,if_verified
0,3646568,USUM71805170,7dt6x5M1jzdTEt8oCbisTK,Better Now,Post Malone,Better Now,1
1,2942139,USUM71710087,7wGoVu4Dady5GV0Sv4UIsx,rockstar,Post Malone,​​rockstar,1
2,2992529,USSM11706905,0ofbQMrRDsUaVKq2mGLEAb,Havana,Camila Cabello,Havana,1
3,3188009,USRC11701901,7vGuf3Y35N4wmASOKLUVVU,Silence,Marshmello,Silence,1
4,3430186,USUG11800208,3ee8Jmje8o58CHK66QrVC2,SAD!,XXXTENTACION,SAD!,1


In [8]:
for i in range(10):
    name= df_left['SPOTIFY_ID'][i]+'.' + df_left['TITLE'][i] + '.' + df_left['ARTIST'][i] +'.mp3'
    print(name)

7dt6x5M1jzdTEt8oCbisTK.Better Now.Post Malone.mp3
7wGoVu4Dady5GV0Sv4UIsx.rockstar.Post Malone.mp3
0ofbQMrRDsUaVKq2mGLEAb.Havana.Camila Cabello.mp3
7vGuf3Y35N4wmASOKLUVVU.Silence.Marshmello.mp3
3ee8Jmje8o58CHK66QrVC2.SAD!.XXXTENTACION.mp3
7EmGUiUaOSGDnUUQUDrOXC.Wolves.Selena Gomez.mp3
0X8DcetL926BYiPJYstJTc.Lucid Dreams.Juice WRLD.mp3
0JP9xo3adEtGSdUEISiszL.Moonlight.XXXTENTACION.mp3
6uFsE1JgZ20EXyU0JQZbUR.Look What You Made Me Do.Taylor Swift.mp3
6mrKP2jyIQmM0rw6fQryjr.Let You Down.NF.mp3


In [11]:
#!/usr/bin/python

#from googleapiclient.discovery import build
#from apiclient.errors import HttpError
#from oauth2client.tools import argparser

import urllib
from bs4 import BeautifulSoup
import argparse
#import spotify
import json
from io import StringIO
import subprocess
import traceback
from urllib.parse import quote #from python2 to python3

RED     = "\033[31m"
GREEN   = "\033[32m"
BLUE    = "\033[34m"
YELLOW  = "\033[36m"
DEFAULT = "\033[0m"

ACTION  = BLUE + "[+] " + DEFAULT
ERROR   = RED + "[+] " + DEFAULT
OK      =  GREEN + "[+] " + DEFAULT

#=======================
#   Spotify application
#=======================
CLIENT_ID=""
CALL_BACK_URL=""

# Set DEVELOPER_KEY to the API key value from the APIs & auth > Registered apps
# tab of
#   https://cloud.google.com/console
# Please ensure that you have enabled the YouTube Data API for your project.
DEVELOPER_KEY = "REPLACE_ME"
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

def youtube_search(options):
    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
    developerKey=DEVELOPER_KEY)

    # Call the search.list method to retrieve results matching the specified
    # query term.
    search_response = youtube.search().list(
    q=options.q,
    part="id,snippet",
    maxResults=options.max_results
    ).execute()

    videos = []
    channels = []
    playlists = []

    # Add each result to the appropriate list, and then display the lists of
    # matching videos, channels, and playlists.
    for search_result in search_response.get("items", []):
        if search_result["id"]["kind"] == "youtube#video":
            videos.append("%s (%s)" % (search_result["snippet"]["title"],
                                 search_result["id"]["videoId"]))
        elif search_result["id"]["kind"] == "youtube#channel":
            channels.append("%s (%s)" % (search_result["snippet"]["title"],
                                   search_result["id"]["channelId"]))
        elif search_result["id"]["kind"] == "youtube#playlist":
            playlists.append("%s (%s)" % (search_result["snippet"]["title"],
                                    search_result["id"]["playlistId"]))

    print("Videos:\n", "\n".join(videos), "\n")
    print("Channels:\n", "\n".join(channels), "\n")
    print("Playlists:\n", "\n".join(playlists), "\n")

def searchYoutube(trackname):
    textToSearch = trackname
    query = quote(textToSearch) #from python2 to python3
    url = "https://www.youtube.com/results?search_query=" + query
    response = urllib.request.urlopen(url)
    html = response.read()
    soup = BeautifulSoup(html, "html.parser")
    #we return the first result
    return "https://youtube.com" + soup.findAll(attrs={'class':'yt-uix-tile-link'})[0]['href']

def getTrackName(id, access_token):
    """ get the spotify track name from id """
    print(ACTION + " getting track name")
    proc = subprocess.Popen('curl -sS -X GET "https://api.spotify.com/v1/tracks/'+ id +'?market=ES" -H "Authorization: Bearer '+ access_token +'"', shell=True, stdout=subprocess.PIPE)
#     print(proc)
    tmp = proc.stdout.read()
    #convert from json to string
    #io = StringIO()
    #json.dump(tmp, io)
    data = json.loads(tmp)
    if 'error' in data:
        print(ERROR + "can't found song name")
        print(ERROR + data['error']['message'])
        return None
    else:
        print(OK + "name is " + data["name"])
        return data["name"]

def genUrl():
    """ gen url for getting access token """
    print(ACTION + " generating url for access token")
    print(OK +  "https://accounts.spotify.com/authorize?client_id="+ CLIENT_ID + "&response_type=token&redirect_uri=" + CALL_BACK_URL)

def getAccessToken():
    """ get access token """
    print(ACTION + " getting access token")
    proc = subprocess.Popen('curl -sS -X GET "https://accounts.spotify.com/authorize?client_id='+ CLIENT_ID +'&response_type=token&redirect_uri='+ CALL_BACK_URL +'" -H "Accept: application/json"', shell=True, stdout=subprocess.PIPE)
    tmp = proc.stdout.read()
    data = json.loads(tmp)

    print(data)

def downloadYoutube(link):
    """ downloading the track """
    print(ACTION + "downloading song ..")
    proc = subprocess.Popen('youtube-dl --extract-audio --audio-format mp3 '+ link, shell=True, stdout=subprocess.PIPE)
    tmp = proc.stdout.read()
    
    print(OK + "Song Downloaded")

def header():
    """ header informations """
    print (RED + "@ spotify-dl.py version 0.0.1")
    print (YELLOW + "@ author : Naper")
    print (BLUE + "@ Designed for OSx/linux")
    print ("" + DEFAULT)


pwd = os.getcwd()
newlist = []
access_token = 'BQASgCHrnCn7EprEfGa9I5zTJEh7U1a3wOL08qNsuvFUbGqUpD7f9ry0NrhSILfMWprbIDC5sTutA3dpoYRRWEgKugMeQm59j-VvYiEkFQLPLonjfXzbtdPxubdWEEdy3gx-yCAGGjeF45VilCzJ9dv5B7pzZtpCSLC3FBO6J6mQIEaAfYJ5137a'
j=0
for i in range(1):
    try:        
        oldlist = [f for f in os.listdir(".") 
           if f not in ['scrape.ipynb','.DS_Store','.ipynb_checkpoints'] and 
           os.path.isfile(os.path.join(pwd,f)) and not f.startswith('.')]
        
        track = df_left['TITLE'][i] + ' ' + df_left['ARTIST'][i]
#         name = getTrackName(track, access_token)
        link = searchYoutube(track)
        print(link)
        downloadYoutube(link)
        
        newlist = [f for f in os.listdir(".") 
           if f not in ['scrape.ipynb','.DS_Store','.ipynb_checkpoints'] and 
           os.path.isfile(os.path.join(pwd,f)) and not f.startswith('.')]
        newsong = [item for item in newlist if item not in oldlist][0]
        newname = df_left['ISRC'][i]+'.' + df_left['TITLE'][i] + '.' + df_left['ARTIST'][i] +'.mp3'
        print(newsong, newname)
        os.rename(newsong, newname)
        print(j)
        
        
        
    except:
        print (ERROR + "use --help for help")
        print(i)
        continue 
    j+=1
    

https://youtube.com/watch?v=YQHsXMglC9A
[+] downloading song ..
[+] Song Downloaded
[+] use --help for help
0
